In [156]:
import pandas as pd
import os
import re
from datetime import datetime as dt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Merging the Data to a single DF

In [190]:
def find_all_files(path, ty='csv', Name='Merged_DF.csv'):
    #Iteratively appends all files with ty extention to list_of_files
    for root,dirs,files in os.walk(path):
        [list_of_files.append(file) for file in files if (file.endswith(f".{ty}") and (file!=Name))]

In [191]:
Name = 'Merged_DF.csv' #Name of Final DF
list_of_files = []
data_path = os.path.join('..', 'Data')
find_all_files(data_path, Name=Name)

Total_DF = pd.DataFrame()
for file in list_of_files:
    try:
        DF = pd.read_csv(os.path.join('..', 'Data', file), encoding='utf-8')
    except:
        DF = pd.read_csv(os.path.join('..', 'Data', file), encoding='latin1')
    Total_DF = (DF if Total_DF.empty else pd.concat([Total_DF, DF]))

# Total DF Cleaning

In [192]:
#Total_DF['video_id'].map(lambda x: type(x)!=str).sum() #All values are string
#Total_DF['trending_date'].map(lambda x: type(x)!=str).sum() #All values are string
#Convert the 'trending_date' to date format
Total_DF['trending_date'] = Total_DF['trending_date'].map(lambda x: dt.strptime(x, "%y.%d.%m"))

#Total_DF['title'].map(lambda x: type(x)!=str).sum() #All values are string
#Total_DF['channel_title'].map(lambda x: type(x)!=str).sum() #All values are string
#Total_DF['category_id'].map(lambda x: type(x)!=int).sum() #All values are int
#Total_DF['publish_time'].map(lambda x: type(x)!=str).sum() #All values are string
#Convert the 'publish_time' to date format
Total_DF['publish_time'] = pd.to_datetime(Total_DF['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ') #%f means microsecond which means 6 digits. This works here as it is always 0 microseconds
#Total_DF['tags'].map(lambda x: type(x)!=str).sum() #All values are string
#Total_DF['views'].map(lambda x: type(x)!=int).sum() #All values are int64
#Total_DF['likes'].map(lambda x: type(x)!=int).sum() #All values are int64
#Total_DF['dislikes'].map(lambda x: type(x)!=int).sum() #All values are int64
#Total_DF['comment_count'].map(lambda x: type(x)!=int).sum() #All values are int64
#Total_DF['likes'].isnull().sum()
#Total_DF['dislikes'].isnull().sum()
#Total_DF['comment_count'].isnull().sum()
#Total_DF['thumbnail_link'].isnull().sum()
#Total_DF['comments_disabled'].map(lambda x: type(x)!=bool).sum() #All values are boolean
#Total_DF['ratings_disabled'].map(lambda x: type(x)!=bool).sum() #All values are boolean
#Total_DF['video_error_or_removed'].map(lambda x: type(x)!=bool).sum() #All values are boolean
#Total_DF['comments_disabled'].isnull().sum()
#Total_DF['ratings_disabled'].isnull().sum()
#Total_DF['video_error_or_removed'].isnull().sum()

#Convert NaN values in 'description' to ''
Total_DF['description'].fillna(value='', inplace=True)
#Total_DF['description'].isna().sum()
#Total_DF[Total_DF['description'] == ''].shape

# Removing for duplicate rows

In [250]:
#Drop the duplicate rows
Dup_Rem_DF = Total_DF.drop_duplicates(subset=['video_id', 'trending_date'], keep='last')
Dup_Rem_DF = Dup_Rem_DF.iloc[Dup_Rem_DF[(Dup_Rem_DF['video_id']!='#NAME?')].index, :]
Dup_Rem_DF.shape
#375942 - 52212 - 205 = 323525

(323525, 16)

# Save the DataFrame as pickle 

In [254]:
Dup_Rem_DF.to_pickle("../Data/VideoDF.pkl")

# Read from pickle

In [255]:
Video_DF = pd.read_pickle("../Data/VideoDF.pkl")

In [283]:
Video_DF.head(100)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,SbOwzAl9ZfQ,2017-11-14,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,2017-11-13 06:06:22,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,https://i.ytimg.com/vi/SbOwzAl9ZfQ/default.jpg,False,False,False,Disfruta la presencia del Chef Torreblanca en ...
1,klOV6Xh-DnI,2017-11-14,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,2017-11-13 05:11:58,La Voz Mexico 7,104972,271,174,369,https://i.ytimg.com/vi/klOV6Xh-DnI/default.jpg,False,False,False,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...
2,6L2ZF7Qzsbk,2017-11-14,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,2017-11-13 17:00:02,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,https://i.ytimg.com/vi/6L2ZF7Qzsbk/default.jpg,False,False,False,La canciÃ³n del principio se llama âEste esp...
3,hcY52MFWMDM,2017-11-14,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,2017-11-13 03:47:10,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,https://i.ytimg.com/vi/hcY52MFWMDM/default.jpg,False,False,False,El video es de un Walmart en el pais centroame...
4,_OXDcGPVAa4,2017-11-14,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,2017-11-13 19:17:48,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,https://i.ytimg.com/vi/_OXDcGPVAa4/default.jpg,False,False,False,MI HERMANO NARRA MI RUTINA DE MAQUILLAJE\nhttp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,YnQoEHHM_xc,2017-11-14,ExatlÃ³n MÃ©xico | Programa 20,ExatlÃ³n,24,2017-11-12 05:00:03,"Mexico en alerta|""tv azteca""|""mexico en alerta...",377398,2378,289,1883,https://i.ytimg.com/vi/YnQoEHHM_xc/default.jpg,False,False,False,Programa completo del 11 de noviembre de 2017 ...
111,ExbVoHLXPzg,2017-11-14,Â¿Por quÃ© se cree en supersticiones? - Curios...,CuriosaMente,27,2017-11-12 14:00:02,"ciencia|""niÃ±os""|""divulgaciÃ³n""|""science""|""cur...",129904,10161,124,543,https://i.ytimg.com/vi/ExbVoHLXPzg/default.jpg,False,False,False,Los humanos somos mÃ¡s supersticiosos de lo qu...
112,eYcP7ItiHHI,2017-11-14,Mandando saludos,Raymix,10,2017-11-13 03:22:33,[none],21728,3169,86,384,https://i.ytimg.com/vi/eYcP7ItiHHI/default.jpg,False,False,False,
113,s_kRjjlGCcQ,2017-11-14,Clash Royale: Crown Championship LATAM Finais ...,Clash Royale Brasil,20,2017-11-13 02:43:41,"mobile|""celular""|""clash""|""royale""|""supercell""|...",179904,9811,294,436,https://i.ytimg.com/vi/s_kRjjlGCcQ/default.jpg,False,False,False,Entre na Arena! Dos criadores do Clash of Clan...
